In [1]:
# LangChain 및 문서 로더
!pip install langchain langchain-community

# 문서 처리용
!pip install python-docx
!pip install python-pptx
!pip install openpyxl
!pip install fpdf

# PDF 로더
!pip install pypdf

In [2]:
# LangChain
from langchain.docstore.document import Document
from langchain_community.document_loaders import (
    TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader, UnstructuredExcelLoader
)

# 파일 처리
import os
import openpyxl
from fpdf import FPDF
from pptx import Presentation
from docx import Document as DocxWriter

In [3]:
############################################
# --- Slack (더미) ---
############################################

def generate_dummy_slack_messages():
    """Slack 더미 메시지 10개 생성"""
    base_messages = [
        {"type": "message", "user": "U10001", "text": "오늘 회의는 오후 3시에 시작합니다.", "ts": "1694958400.000200"},
        {"type": "message", "user": "U10002", "text": "회의 자료는 구글 드라이브에 올려놨어요.", "ts": "1694958600.000300"},
        {"type": "message", "user": "U10003", "text": "네 확인했습니다!", "ts": "1694958700.000400"},
        {"type": "message", "user": "U10004", "text": "프로젝트 일정표 업데이트했습니다.", "ts": "1694958800.000500"},
        {"type": "message", "user": "U10005", "text": "오늘 점심은 12시에 먹어요.", "ts": "1694958900.000600"},
        {"type": "message", "user": "U10006", "text": "고객사 피드백을 공유드립니다.", "ts": "1694959000.000700"},
        {"type": "message", "user": "U10007", "text": "내일 오전에 코드 리뷰 진행하겠습니다.", "ts": "1694959100.000800"},
        {"type": "message", "user": "U10008", "text": "회의실 예약은 제가 해두었습니다.", "ts": "1694959200.000900"},
        {"type": "message", "user": "U10009", "text": "배포는 오후 6시에 시작할 예정입니다.", "ts": "1694959300.001000"},
        {"type": "message", "user": "U10010", "text": "다들 수고 많으셨습니다!", "ts": "1694959400.001100"},
    ]
    return base_messages

def load_dummy_slack_messages():
    docs = []
    try:
        print(f"🔄 (Dummy) Slack 메시지 10개 로드 중...")
        messages = generate_dummy_slack_messages()
        for msg in messages:
            docs.append(Document(page_content=msg["text"], metadata={"source": "slack"}))
        print(f"✅ (Dummy) Slack {len(messages)}개 로드 완료.")
    except Exception as e:
        print(f"❌ Slack 더미 오류: {e}")
    return docs


############################################
# --- Notion (더미) ---
############################################

def generate_dummy_notion_pages():
    """Notion 더미 회의록 5개 생성"""
    return [
        {"title": "회의록 - 2025년 9월 1일", "작성자": "윤소현", "회의내용": "AI 검색 시스템 초기 아키텍처 논의."},
        {"title": "회의록 - 2025년 9월 5일", "작성자": "김민우", "회의내용": "OneDrive 파일 동기화 오류 사례 검토."},
        {"title": "회의록 - 2025년 9월 8일", "작성자": "박준현", "회의내용": "Outlook 메일 데이터를 벡터스토어에 저장하는 구조 설계."},
        {"title": "회의록 - 2025년 9월 12일", "작성자": "서준석", "회의내용": "MS Graph API 인증 문제 및 토큰 만료 처리 방법 협의."},
        {"title": "회의록 - 2025년 9월 15일", "작성자": "윤소현", "회의내용": "Slack-Notion-Graph API 통합 후 테스트 결과 공유."}
    ]

def load_dummy_notion_pages():
    docs = []
    try:
        print("🔄 (Dummy) Notion 회의록 5개 로드 중...")
        dummy_pages = generate_dummy_notion_pages()
        for page in dummy_pages:
            docs.append(
                Document(
                    page_content=f"{page['title']}\n\n내용: {page['회의내용']}",
                    metadata={"source": "notion_dummy", "작성자": page["작성자"]}
                )
            )
        print(f"✅ (Dummy) Notion {len(dummy_pages)}개 로드 완료.")
    except Exception as e:
        print(f"❌ Notion 더미 오류: {e}")
    return docs


############################################
# --- OneDrive (더미) ---
############################################

def generate_dummy_onedrive_files(base_dir="./downloads"):
    """OneDrive 더미 파일 5개 생성"""
    os.makedirs(base_dir, exist_ok=True)
    file_paths = []

    # TXT
    with open(os.path.join(base_dir, "메모.txt"), "w", encoding="utf-8") as f:
        f.write("이것은 테스트용 TXT 파일입니다. " * 10)
    file_paths.append("메모.txt")

    # DOCX
    doc = DocxWriter()
    doc.add_paragraph("이것은 테스트용 DOCX 문서입니다. " * 10)
    doc.save(os.path.join(base_dir, "회의록.docx"))
    file_paths.append("회의록.docx")

    # PDF (한글 폰트 등록)
    pdf = FPDF()
    pdf.add_page()
    font_path = "./fonts/Roboto-VariableFont_wdth,wght.ttf"
    if not os.path.exists(font_path):
        raise FileNotFoundError(f"폰트 파일이 {font_path} 에 없습니다. fonts 디렉토리에 추가해 주세요.")
    pdf.add_font("Roboto", "", font_path, uni=True)
    pdf.set_font("Roboto", size=12)
    pdf.multi_cell(0, 10, "이것은 테스트용 PDF 문서입니다. " * 10)
    pdf.output(os.path.join(base_dir, "연구보고서.pdf"))
    file_paths.append("연구보고서.pdf")

    # PPTX
    prs = Presentation()
    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = "테스트용 PPT 문서"
    slide.placeholders[1].text = "본문 예시입니다."
    prs.save(os.path.join(base_dir, "발표자료.pptx"))
    file_paths.append("발표자료.pptx")

    # XLSX
    wb = openpyxl.Workbook()
    ws = wb.active
    ws["A1"] = "이것은 테스트용 XLSX 문서입니다."
    wb.save(os.path.join(base_dir, "실험데이터.xlsx"))
    file_paths.append("실험데이터.xlsx")

    return file_paths

def load_file_as_documents(file_path: str):
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    if ext == ".txt":
        loader = TextLoader(file_path, encoding="utf-8")
    elif ext == ".pdf":
        loader = PyPDFLoader(file_path)
    elif ext == ".docx":
        loader = UnstructuredWordDocumentLoader(file_path)
    elif ext == ".pptx":
        loader = UnstructuredPowerPointLoader(file_path)
    elif ext == ".xlsx":
        loader = UnstructuredExcelLoader(file_path)
    else:
        return [Document(page_content=f"[Unsupported file type: {ext}]", metadata={"filename": file_path})]
    return loader.load()

def load_dummy_onedrive_files(base_dir="./downloads"):
    docs = []
    print("🔄 (Dummy) OneDrive 파일 5개 로드 중...")
    file_names = generate_dummy_onedrive_files(base_dir=base_dir)
    for name in file_names:
        file_path = os.path.join(base_dir, name)
        file_docs = load_file_as_documents(file_path)
        for d in file_docs:
            d.metadata["source"] = "onedrive"
            d.metadata["filename"] = name
        docs.extend(file_docs)
    print(f"✅ (Dummy) OneDrive {len(docs)}개 Document 로드 완료.")
    return docs


############################################
# --- Outlook (더미) ---
############################################

def generate_dummy_outlook_emails():
    """업무에서 자주 볼 수 있는 Outlook 더미 메일 10개 생성"""
    return [
        {
            "subject": "팀 회의 일정 공지",
            "bodyPreview": "안녕하세요, 내일 오후 3시에 회의실 B에서 팀 회의가 예정되어 있습니다."
        },
        {
            "subject": "월간 보고서 제출 요청",
            "bodyPreview": "이번 달 보고서는 금요일까지 제출 부탁드립니다. 양식은 OneDrive에 업로드 되어 있습니다."
        },
        {
            "subject": "휴가 신청 승인",
            "bodyPreview": "홍길동 님의 9월 20일~9월 22일 휴가 신청이 승인되었습니다."
        },
        {
            "subject": "시스템 점검 안내",
            "bodyPreview": "이번 주 토요일 00시~06시 사이에 이메일 서버 정기 점검이 예정되어 있습니다."
        },
        {
            "subject": "고객사 미팅 일정 확인",
            "bodyPreview": "고객사 ABC와의 미팅이 9월 25일 오전 10시에 진행됩니다. 장소: 본사 회의실 A."
        },
        {
            "subject": "비용 정산 안내",
            "bodyPreview": "출장비 내역을 이번 주 안에 ERP 시스템에 업로드 부탁드립니다."
        },
        {
            "subject": "보안 교육 이수 안내",
            "bodyPreview": "정보 보안 교육은 9월 30일까지 온라인 플랫폼에서 이수 완료해야 합니다."
        },
        {
            "subject": "신규 프로젝트 착수 보고",
            "bodyPreview": "신규 프로젝트 'AI 기반 검색 시스템'이 10월 1일부터 시작됩니다."
        },
        {
            "subject": "연차 사용 안내",
            "bodyPreview": "10월 첫째 주 연휴 기간 연차 사용 계획을 팀 리더에게 공유 부탁드립니다."
        },
        {
            "subject": "회의 자료 공유",
            "bodyPreview": "오늘 회의에서 사용된 발표 자료를 OneDrive 링크로 공유드립니다."
        }
    ]

def load_dummy_outlook_emails():
    docs = []
    try:
        print("🔄 (Dummy) Outlook 메일 10개 로드 중...")
        dummy_emails = generate_dummy_outlook_emails()
        for mail in dummy_emails:
            subject = mail["subject"]
            body_preview = mail["bodyPreview"]
            docs.append(
                Document(page_content=body_preview, metadata={"source": f"outlook:{subject}"})
            )
            print(f"📌 변환 완료 → 제목: {subject}")
        print(f"✅ (Dummy) Outlook {len(dummy_emails)}개 로드 완료.")
    except Exception as e:
        print(f"❌ Outlook 더미 오류: {e}")
    return docs



############################################
# --- 전체 문서 로딩 ---
############################################

def load_all_documents():
    summary = {}
    all_docs = []

    slack_docs = load_dummy_slack_messages()
    summary["Slack"] = len(slack_docs)
    all_docs.extend(slack_docs)

    notion_docs = load_dummy_notion_pages()
    summary["Notion"] = len(notion_docs)
    all_docs.extend(notion_docs)

    onedrive_docs = load_dummy_onedrive_files()
    summary["OneDrive"] = len(onedrive_docs)
    all_docs.extend(onedrive_docs)

    outlook_docs = load_dummy_outlook_emails()
    summary["Outlook"] = len(outlook_docs)
    all_docs.extend(outlook_docs)

    return all_docs, summary

In [4]:
# 실행 예시
if __name__ == "__main__":
    docs, summary = load_all_documents()
    print("\n=== 📊 문서 로딩 결과 요약 ===")
    for source, count in summary.items():
        print(f"{source}: {count}개")
    print(f"총 {len(docs)}개 문서 로드 완료.\n")

    for d in docs[:5]:
        print(f"▶ {d.metadata} | {d.page_content[:50]}...")

🔄 (Dummy) Slack 메시지 10개 로드 중...
✅ (Dummy) Slack 10개 로드 완료.
🔄 (Dummy) Notion 회의록 5개 로드 중...
✅ (Dummy) Notion 5개 로드 완료.
🔄 (Dummy) OneDrive 파일 5개 로드 중...
✅ (Dummy) OneDrive 5개 Document 로드 완료.
🔄 (Dummy) Outlook 메일 10개 로드 중...
📌 변환 완료 → 제목: 팀 회의 일정 공지
📌 변환 완료 → 제목: 월간 보고서 제출 요청
📌 변환 완료 → 제목: 휴가 신청 승인
📌 변환 완료 → 제목: 시스템 점검 안내
📌 변환 완료 → 제목: 고객사 미팅 일정 확인
📌 변환 완료 → 제목: 비용 정산 안내
📌 변환 완료 → 제목: 보안 교육 이수 안내
📌 변환 완료 → 제목: 신규 프로젝트 착수 보고
📌 변환 완료 → 제목: 연차 사용 안내
📌 변환 완료 → 제목: 회의 자료 공유
✅ (Dummy) Outlook 10개 로드 완료.

=== 📊 문서 로딩 결과 요약 ===
Slack: 10개
Notion: 5개
OneDrive: 5개
Outlook: 10개
총 30개 문서 로드 완료.

▶ {'source': 'slack'} | 오늘 회의는 오후 3시에 시작합니다....
▶ {'source': 'slack'} | 회의 자료는 구글 드라이브에 올려놨어요....
▶ {'source': 'slack'} | 네 확인했습니다!...
▶ {'source': 'slack'} | 프로젝트 일정표 업데이트했습니다....
▶ {'source': 'slack'} | 오늘 점심은 12시에 먹어요....
